In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
def add_date_time(df):
    dts = []
    for idx in df.index:
        dts.append( datetime(
            df.loc[idx, 'year'],
            df.loc[idx, 'month'],
            df.loc[idx, 'day'],
            df.loc[idx, 'hour']-1, 
            0 # minutes
        ))
    df['date_time'] = dts
    return df

In [ ]:
demands = {
    'ERCOT' : ['../Input_Data/ReliabilityPaper/ERCOT_mem_1998-2019.csv', 0, 2003],
    'PJM' : ['../Input_Data/ReliabilityPaper/PJM_mem_1993-2019.csv', 0, 2006],
    'NYISO' : ['../Input_Data/ReliabilityPaper/NYISO_demand_unnormalized.csv', 5, 2004],
}

dfs = {}
for k, v in demands.items():
    print(k, v)
    df = pd.read_csv(v[0], header=v[1])
    print(len(df.index))
    df = df[ (df['year'] >= v[2]) & (df['year'] <= 2019) ]
    df = df.reset_index()
    df = add_date_time(df)
    print(len(df.index))
    dfs[k] = df.reset_index()
    print(dfs[k].head())
    print(dfs[k].tail())

In [ ]:
# Based on common color blindness
# https://www.nature.com/articles/nmeth.1618/figures/2
# Skip black and reserve it for other specific lines
def color_list():
    l = [
            np.array([230, 159, 0]), # orange
            np.array([86, 180, 233]), # Sky blue
            np.array([0, 158, 115]), # Bluish green
            np.array([240, 228, 66]), # Yellow
            np.array([0, 114, 178]), # Blue
            np.array([213, 94, 0]), # Vermillion
            np.array([204, 121, 167]), # Reddish purple
    ]
    return [i/255. for i in l]

def fit(k, v):
    matplotlib.rcParams.update({'font.size': 20})
    plt.close()
    fix, ax = plt.subplots(figsize=(15,5))
    print(k)#, v)
    x = v.index
    y = v['demand (MW)']
    dt = v['date_time']
    coef = np.polyfit(x, y, 1)
    poly1d_fn = np.poly1d(coef) 
    # poly1d_fn is now a function which takes in x and returns an estimate for y
    print(coef)
    print(f"Delta 1 year: {coef[0]*8760}")
    print(f"Delta 1 year as % yr 1: {coef[0]*8760/coef[1]}")
    
    ax.plot(dt,y, 'y.')
    ax.plot(dt, poly1d_fn(x), '--k')
    
    ax.set_ylabel('load (MW)')
    ax.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter('{x:,.0f}'))
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    plt.setp( ax.xaxis.get_majorticklabels(), rotation=45 )
    
    ax.set_ylim(0, ax.get_ylim()[1])
    plt.tight_layout()
    plt.savefig(f'raw_demand_{k}.pdf')
    plt.savefig(f'raw_demand_{k}.png')
    

def fit_all(dfs):
    matplotlib.rcParams.update({'font.size': 22})
    plt.close()
    fix, axs = plt.subplots(nrows=3, ncols=1, figsize=(15,10), sharex=True)
    
    colors = color_list()
    i = 0
    for k, ax in zip(["PJM", "ERCOT", "NYISO"], axs):
        print(i, k)
        x = dfs[k].index
        y = dfs[k]['demand (MW)']
        dt = dfs[k]['date_time']
        coef = np.polyfit(x, y, 1)
        poly1d_fn = np.poly1d(coef) 
        # poly1d_fn is now a function which takes in x and returns an estimate for y
        print(coef)
        print(f"Delta 1 year as % yr 1: {coef[0]*8760/coef[1]*100.}")

    
        ax.plot(dt,y, c=colors[i], marker='.', label='_nolabel_')
        ax.plot(dt, poly1d_fn(x), '--k', label='_nolabel_')
    
        if i == 2:
            ax.xaxis.set_major_locator(mdates.YearLocator())
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
            ax.xaxis.set_minor_locator(mdates.MonthLocator())
        
        ax.set_ylabel('load (MW)')
        ax.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter('{x:,.0f}'))
        ax.set_ylim(0, ax.get_ylim()[1])
        if i == 0:
            j = 0
            for k in ["PJM", "ERCOT", "NYISO"]:
                axs[0].scatter([], [], c=colors[j], marker='o', label=k)
                j += 1
        i += 1
    
   
    axs[0].legend(loc='upper left')
    
    plt.setp( axs[2].xaxis.get_majorticklabels(), rotation=45 )
    plt.tight_layout()
    plt.subplots_adjust(hspace = 0.0)
    plt.savefig('raw_demand_ALL.pdf')
    plt.savefig('raw_demand_ALL.png')

#for k, v in dfs.items():
#    print(f"\n\n{k}")
#    fit(k, v)
    
#    print(f"Mean 2019 demand: {np.mean(v.loc[ (v['year']==2019), 'demand (MW)'])}")

fit_all(dfs)